In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split

from utils import data_preprocessing_util as dpu
from utils import classification_util as cu

from MLP import MLP
from nn_framework import NNFramework

import seaborn as sns
import matplotlib.pyplot as plt

# Fertility Dataset

## Preprocessing

In [4]:
data_path = r'data/fertility_diagnosis.csv'
df = pd.read_csv(data_path)
df = dpu.preprocess_fertility_dataset(df)

In [5]:
nn = NNFramework()
nn.fit_encoder(df=df, cols_to_encode=df.columns.difference(['age', 'hours_sitting']))
df_encoded = nn.encode_dataset(df)

In [6]:
len(df_encoded.columns)

21

## Parameter experiments

In [7]:
random_seed = 1038
scaling = True
oversampling = True

scaler = preprocessing.StandardScaler() if scaling else None

In [8]:
y = df_encoded['diagnosis']
X = df_encoded[df_encoded.columns.difference(['diagnosis'])]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = random_seed, shuffle=True, stratify=y)


Params to check out:
- 2 activation functions
- 3 lrs: 0.001, 0.01, 0.1
- number of nodes per layer
- 1 layer:
    - 5
    - 32
- 2 layer:
    - 16 16
- 3 layers
    - 10 5 5
    - 16 8 8
    - 64 32 32

In [43]:
activation_functions = ['relu', 'sigmoid']
learning_rates = [0.0001, 0.001, 0.01, 0.1, 0.5]
hidden_layer_sizes = [(5,), (32,), (16, 16), (10, 5, 5), (16, 8, 8), (64, 32, 32),]

In [49]:
methods = []

for af in activation_functions:
    for lr in learning_rates:
        for hls in hidden_layer_sizes:
            methods.append((f'MLP-{af}-{lr}-{hls}', MLP(n_iter=5000, activation_function=af, learning_rate=lr, hidden_layer_sizes=hls)))
    
pipelines = cu.define_pipelines(methods, scaler=scaler, oversampling=oversampling)

In [41]:
import warnings
warnings.filterwarnings("ignore")

In [50]:
cv_num = 5
model_params = {}
models = {}

for model_name, pipeline in pipelines.items():
    cv_results = cross_validate(pipeline, X, y, cv=cv_num, scoring='f1_macro', return_estimator=True)

    models[model_name] = cv_results['estimator']
    model_params[model_name] = {}

    num_cols = ['test_score', 'fit_time', 'score_time']

    for num_col in num_cols:
        model_params[model_name][num_col] = cv_results[num_col]
        model_params[model_name][f'{num_col}_mean'] = cv_results[num_col].mean()
        model_params[model_name][f'{num_col}_std'] = cv_results[num_col].std()
    
    model_params[model_name]['parameter_num'] = cv_results['estimator'][0][model_name].number_of_params_
    model_params[model_name]['hidden_layer_sizes'] = cv_results['estimator'][0][model_name].hidden_layer_sizes
    model_params[model_name]['activation_function'] = cv_results['estimator'][0][model_name].activation_function
    model_params[model_name]['learning_rate'] = cv_results['estimator'][0][model_name].learning_rate
    model_params[model_name]['converged'] = [e[model_name].converged_ for e in cv_results['estimator']]
    model_params[model_name]['validation_losses'] = [e[model_name].validation_losses_ for e in cv_results['estimator']]
    model_params[model_name]['training_losses'] = [e[model_name].training_losses_ for e in cv_results['estimator']]
    model_params[model_name]['num_iter'] = np.array(list([len(e[model_name].training_losses_) for e in cv_results['estimator']])).mean()

    print(model_name)
    print(
        f"f1 scores: {model_params[model_name]['test_score']}\n" +
        f"f1 mean: {model_params[model_name]['test_score_mean']:.3f}\n" +
        f"f1 std: {model_params[model_name]['test_score_std']:.3f}\n"
    )
    print('----------------------------------------------------------------------------------------------------')
    

MLP-relu-0.001-(5,)
f1 scores: [0.39393939 0.56709957 0.60784314 0.49820789 0.44444444]
f1 mean: 0.502
f1 std: 0.078

----------------------------------------------------------------------------------------------------
MLP-relu-0.001-(32,)
f1 scores: [0.49820789 0.56709957 0.72222222 0.42857143 0.60784314]
f1 mean: 0.565
f1 std: 0.100

----------------------------------------------------------------------------------------------------
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized weights.
MLP-relu-0.001-(16, 16)
f1 scores: [0.09090909 0.09090909 0.09090909 0.41176471 0.39393939]
f1 mean: 0.216
f1 std: 0.153

----------------------------------------------------------------------------------------------------
As a punishment we return a model with randomly initialized weights.
As a punishment we return a model with randomly initialized we

In [82]:
df_param = pd.DataFrame(model_params).transpose()
df_param = df_param.reset_index(drop=False)
df_param = df_param.rename(columns={'index': 'model'})

df_param

,model,test_score,test_score_mean,test_score_std,fit_time,fit_time_mean,fit_time_std,score_time,score_time_mean,score_time_std,parameter_num,hidden_layer_sizes,activation_function,learning_rate,converged,validation_losses,training_losses,num_iter
0,"MLP-relu-0.001-(5,)","[0.39393939393939387, 0.5670995670995671, 0.60...",0.502307,0.078031,"[5.800848960876465, 5.833294153213501, 5.65875...",5.771315,0.060739,"[0.0020003318786621094, 0.0029938220977783203,...",0.002199,0.000397,117,"(5,)",relu,0.001,"[True, True, True, True, True]","[[1.2334464780565, 0.9929382153109731, 0.83348...","[[0.9495062401575112, 0.8133720555444218, 0.73...",1000.0
1,"MLP-relu-0.001-(32,)","[0.4982078853046595, 0.5670995670995671, 0.722...",0.564789,0.099632,"[6.034198999404907, 6.041671991348267, 5.99051...",6.031088,0.032892,"[0.003000497817993164, 0.0020008087158203125, ...",0.002601,0.00049,738,"(32,)",relu,0.001,"[True, True, True, True, True]","[[0.571559500828169, 0.46260801340678004, 0.40...","[[0.6713400062412406, 0.5928890608372823, 0.54...",1000.0
2,"MLP-relu-0.001-(16, 16)","[0.09090909090909091, 0.09090909090909091, 0.0...",0.215686,0.152924,"[2.7233526706695557, 4.19177770614624, 2.14267...",5.075912,2.605945,"[0.002000093460083008, 0.002000570297241211, 0...",0.0024,0.00049,642,"(16, 16)",relu,0.001,"[False, False, False, True, True]","[[0.7202748805536567, 0.5960108189752704, 0.54...","[[0.856512505048089, 0.747793020727742, 0.6845...",621.2
3,"MLP-relu-0.001-(10, 5, 5)","[0.09090909090909091, 0.09090909090909091, 0.0...",0.238329,0.180552,"[0.283064603805542, 0.24588608741760254, 0.348...",0.27296,0.048594,"[0.002000570297241211, 0.003001689910888672, 0...",0.002201,0.000401,307,"(10, 5, 5)",relu,0.001,"[False, False, False, False, False]","[[0.8999277271121642, 0.9042320696322854, 0.91...","[[0.9860105884175189, 0.9891275796274209, 0.99...",25.4
4,"MLP-relu-0.001-(16, 8, 8)","[0.4736842105263158, 0.4736842105263158, 0.473...",0.336384,0.168157,"[0.031006574630737305, 0.054012298583984375, 0...",0.040212,0.012123,"[0.0020003318786621094, 0.002000570297241211, ...",0.002401,0.00049,562,"(16, 8, 8)",relu,0.001,"[False, False, False, False, False]","[[4.134041693741337, 10.812845872129204], [1.8...","[[4.537235663629631, nan], [1.7955302805772584...",3.0
5,"MLP-relu-0.001-(64, 32, 32)","[0.4594594594594595, 0.4594594594594595, 0.473...",0.462304,0.00569,"[0.004259824752807617, 0.00500178337097168, 0....",0.004854,0.000297,"[0.002999544143676758, 0.0018682479858398438, ...",0.002374,0.000513,4546,"(64, 32, 32)",relu,0.001,"[False, False, False, False, False]","[[], [], [], [], []]","[[], [], [], [], []]",0.0
6,"MLP-relu-0.01-(5,)","[0.09090909090909091, 0.09090909090909091, 0.0...",0.249684,0.196387,"[3.579591989517212, 3.7283995151519775, 3.0883...",4.37326,1.132105,"[0.002000570297241211, 0.002000570297241211, 0...",0.002401,0.00049,117,"(5,)",relu,0.01,"[False, False, False, True, True]","[[0.5288148363661361, 0.41737921156972974, 0.3...","[[0.5893710807460183, 0.512832762659213, 0.452...",762.2
7,"MLP-relu-0.01-(32,)","[0.09090909090909091, 0.19999999999999998, 0.0...",0.274989,0.212408,"[2.231398582458496, 2.6350533962249756, 1.8674...",3.826858,1.952976,"[0.0020003318786621094, 0.0020008087158203125,...",0.002,0.000001,738,"(32,)",relu,0.01,"[False, False, False, True, True]","[[0.24396183892380904, 0.1713048913483235, 0.1...","[[0.4532890628303428, 0.35828773587267687, 0.2...",625.6
8,"MLP-relu-0.01-(16, 16)","[0.09090909090909091, 0.09090909090909091, 0.0...",0.238329,0.180552,"[0.013003110885620117, 0.016004085540771484, 0...",0.013203,0.002926,"[0.0020008087158203125, 0.002000093460083008, ...",0.002001,0.0,642,"(16, 16)",relu,0.01,"[False, False, False, False, False]","[[84.07095085701769], [nan], [4.82252291446771...","[[nan], [nan], [7.647845614030195], [], [nan]]",0.8
9,"MLP-relu-0.01-(10, 5, 5)","[0.09090909090909091, 0.09090909090909091, 0.0...",0.238329,0.180552,"[0.009002685546875, 0.009002208709716797, 0.00...",0.007202,0

In [83]:
df_param.to_csv(r'results/fertility_params.csv', index=False)

In [84]:
df_param.loc[:,['activation_function', 'converged', 'learning_rate', 'hidden_layer_sizes', 'num_iter']]

,activation_function,converged,learning_rate,hidden_layer_sizes,num_iter
0,relu,"[True, True, True, True, True]",0.001,"(5,)",1000.0
1,relu,"[True, True, True, True, True]",0.001,"(32,)",1000.0
2,relu,"[False, False, False, True, True]",0.001,"(16, 16)",621.2
3,relu,"[False, False, False, False, False]",0.001,"(10, 5, 5)",25.4
4,relu,"[False, False, False, False, False]",0.001,"(16, 8, 8)",3.0
5,relu,"[False, False, False, False, False]",0.001,"(64, 32, 32)",0.0
6,relu,"[False, False, False, True, True]",0.01,"(5,)",762.2
7,relu,"[False, False, False, True, True]",0.01,"(32,)",625.6
8,relu,"[False, False, False, False, False]",0.01,"(16, 16)",0.8
9,relu,"[False, False, False, False, False]",0.01,"(10, 5, 5)",0.0


## Comparison 